In [1]:
#1.3 ellipsoid
# Let M be an ellipsoid in R3, (ax)^{2}+(by)^{2}+z^{2}=d
# x = 1/a*cos(p)*sin(t)*e1 + 1/b*sin(p)*sin(t)*e2 + cos(t)*e3, t, p in R

from gc_utils import *
alg = Algebra(3)
locals().update(alg.blades)

c2e = lambda p, t: 4*np.cos(p)*np.sin(t)*e1 + 5*np.sin(p)*np.sin(t)*e2 + np.cos(t)*e3
e_p = lambda p, t: -4*np.sin(p)*np.sin(t)*e1 + 5*np.cos(p)*np.sin(t)*e2
e_t = lambda p, t: (4*np.cos(p)*np.cos(t)*e1) + (5*np.sin(p)*np.cos(t)*e2) - (np.sin(t)*e3)
def Ix(x):
    p = np.arctan((4/5)*x.e2/x.e1)
    t = np.arccos(x.e3)
    return e_p(p, t) ^ e_t(p, t)

# |p| < pi/2, 0 < t < pi for unique arctan & arccos
p, t = -1, 3
x = c2e(p, t)
I = Ix(x)
nIx = lambda x: normalize(Ix(x))
nI = nIx(x)

nI, nI ^ (3*e_t(p,t) + 5*e_p(p,t))

(1.0 𝐞₁₂ + -0.024 𝐞₁₃ + -0.0192 𝐞₂₃, 1.29e-15 𝐞₁₂₃)

In [2]:
#5.1
C, D, E = [random_multivector(alg) for _ in range(3)]
V1, V2 = [random_versor(2, alg) for _ in range(2)]
f = lambda x: V1.sw(inv(x)) + V2.sw(x**3)
A = lambda x: E*x**2 + C*x*D
a = lambda x: A(x).grade(1)
PA = lambda x: P(A(x), Ix(x))
pa = lambda x: P(a(x), Ix(x))
f_ = lambda a: lambda x: differential(f, x, a)

x, f(x), pa(x), f_(pa(x))(x)

(0.305 𝐞₁ + -0.594 𝐞₂ + -0.99 𝐞₃,
 0.359 𝐞₁ + -2.56 𝐞₂ + -1.14 𝐞₃,
 -0.183 𝐞₁ + -0.323 𝐞₂ + 0.00421 𝐞₃,
 -0.0976 𝐞₁ + -0.949 𝐞₂ + 0.507 𝐞₃)

In [3]:
#5.2 For tangent fields, the outermorphism can be computed in the embedded space
# the scalar product in the skew_symmetrizer acts as a projection into the tangent space
def outermorphism(f, field, alg, h=1e-6, frame=None, r_frame=None, Ix=None):
    def f_outer(x):
        nonlocal frame, r_frame
        if Ix:
            frame = blade_split(Ix(x), alg)
            r_frame = reciprocal(frame)
        A = field(x)
        wf = lambda vectors: wedge([f(v)(x) for v in vectors])
        outer = 0
        for r in A.grades:
            if r == 0:
                outer += A.grade(0)
                continue
            outer += skew_symmetrizer(wf, A.grade(r), alg, h, frame, r_frame)
        return outer
    return f_outer

outermorphism(f_, A, alg)(x), outermorphism(f_, A, alg, Ix=Ix)(x), outermorphism(f_, PA, alg)(x), outermorphism(f_, PA, alg, Ix=Ix)(x)

(-0.434 + -0.0435 𝐞₁ + -2.51 𝐞₂ + -0.313 𝐞₃ + 9.01 𝐞₁₂ + 5.61 𝐞₁₃ + -3.07 𝐞₂₃ + 12.0 𝐞₁₂₃,
 -0.434 + -0.0976 𝐞₁ + -0.949 𝐞₂ + 0.507 𝐞₃ + -0.798 𝐞₁₂ + 0.366 𝐞₁₃ + -0.593 𝐞₂₃ + -4.62e-18 𝐞₁₂₃,
 -0.434 + -0.0977 𝐞₁ + -0.949 𝐞₂ + 0.507 𝐞₃ + -0.798 𝐞₁₂ + 0.366 𝐞₁₃ + -0.593 𝐞₂₃,
 -0.434 + -0.0976 𝐞₁ + -0.949 𝐞₂ + 0.507 𝐞₃ + -0.798 𝐞₁₂ + 0.366 𝐞₁₃ + -0.593 𝐞₂₃ + -4.62e-18 𝐞₁₂₃)

In [4]:
#5.3
f_vec = lambda a: lambda x: differential(f, x, a)
f_ = lambda A: outermorphism(f_vec, A, alg, Ix=Ix)
I1 = f_(Ix)(x)
fnI = f_(nIx)(x)
norm(I1)/norm(I), norm(fnI)

(9.401843494779426, 9.401843494779426)

In [5]:
# Jacobian
Jf = lambda x: norm(f_(nIx)(x))

In [6]:
#5.4
def adjoint_outermorphism(f, field, alg, h=1e-6, frame=None, r_frame=None, Ix=None):
    def f_outer(x):
        A = field(x)
        nonlocal frame, r_frame
        if Ix:
            frame = blade_split(Ix(x), alg)
            r_frame = reciprocal(frame)
        F = lambda vectors: wedge([f(v)(x) for v in vectors]).sp(A)
        outer = 0
        for r in A.grades:
            if r == 0:
                outer += A.grade(0)
                continue
            # Why np.zeros(r)? The derivative of a linear function is constant
            outer += simplicial_derivative(F, np.zeros(r), alg, h, frame, r_frame)
        return outer
    return f_outer

def adjoint_outermorphism_(f, A, alg, h=1e-6, frame=None, r_frame=None, Ix=None):
    def f_outer(x):
        nonlocal frame, r_frame
        if Ix:
            frame = blade_split(Ix(x), alg)
            r_frame = reciprocal(frame)
        F = lambda vectors: wedge([f(v)(x) for v in vectors]).sp(A)
        outer = 0
        for r in A.grades:
            if r == 0:
                outer += A.grade(0)
                continue
            # Why np.zeros(r)? The derivative of a linear function is constant
            outer += simplicial_derivative(F, np.zeros(r), alg, h, frame, r_frame)
        return outer
    return f_outer

adjoint_outermorphism(f_vec, A, alg, Ix=Ix)(x)

-0.434 + 0.314 𝐞₁ + -0.713 𝐞₂ + 0.0232 𝐞₃ + -5.06 𝐞₁₂ + 0.121 𝐞₁₃ + 0.0973 𝐞₂₃

In [7]:
240/(2*10*200/1000*7.3), 100/ (np.pi*49/4), 130/100*5

(8.219178082191782, 2.598448050479924, 6.5)

In [8]:
#5.10a
f_(A)(x), f_(PA)(x), P(f_(A)(x), I1)

(-0.434 + -0.0976 𝐞₁ + -0.949 𝐞₂ + 0.507 𝐞₃ + -0.798 𝐞₁₂ + 0.366 𝐞₁₃ + -0.593 𝐞₂₃ + -4.62e-18 𝐞₁₂₃,
 -0.434 + -0.0976 𝐞₁ + -0.949 𝐞₂ + 0.507 𝐞₃ + -0.798 𝐞₁₂ + 0.366 𝐞₁₃ + -0.593 𝐞₂₃ + -4.62e-18 𝐞₁₂₃,
 -0.434 + -0.0976 𝐞₁ + -0.949 𝐞₂ + 0.507 𝐞₃ + -0.798 𝐞₁₂ + 0.366 𝐞₁₃ + -0.593 𝐞₂₃)

In [9]:
#5.10b
P_ = lambda A: lambda x: P(A(x), I)
P__ = lambda A: lambda x: P(A(x), I1)
fa = lambda A: adjoint_outermorphism(f_vec, A, alg, Ix=Ix)
fa(A)(x), fa(P__(A))(x), P_(fa(A))(x)

(-0.434 + 0.314 𝐞₁ + -0.713 𝐞₂ + 0.0232 𝐞₃ + -5.06 𝐞₁₂ + 0.121 𝐞₁₃ + 0.0973 𝐞₂₃,
 -0.434 + 0.314 𝐞₁ + -0.713 𝐞₂ + 0.0232 𝐞₃ + -5.06 𝐞₁₂ + 0.121 𝐞₁₃ + 0.0973 𝐞₂₃,
 -0.434 + 0.314 𝐞₁ + -0.713 𝐞₂ + 0.0232 𝐞₃ + -5.06 𝐞₁₂ + 0.121 𝐞₁₃ + 0.0973 𝐞₂₃)

In [10]:
#5.12 outermorphism push forward tangent vectors
G = lambda x: (x+D)**2
F = lambda x: G(f(x))
differential(F, x, pa(x)), differential(G, f(x), f_(pa)(x))

(2.33 + -0.0953 𝐞₁ + -0.925 𝐞₂ + 0.495 𝐞₃ + 0.247 𝐞₁₂ + 0.463 𝐞₁₃ + -0.0476 𝐞₂₃ + 0.599 𝐞₁₂₃,
 2.33 + -0.0953 𝐞₁ + -0.925 𝐞₂ + 0.495 𝐞₃ + 0.248 𝐞₁₂ + 0.463 𝐞₁₃ + -0.0476 𝐞₂₃ + 0.599 𝐞₁₂₃)

In [12]:
#5.12c adjoint-outermorphism is the pullback/chain rule of derivatives
# without explicit define the inverse of f, we push forward the frame

def f_derivative_gen(x, f, alg, Ix, h=1e-6):
    f_vec = lambda a: lambda x: differential(f, x, a)
    f_ = lambda A: outermorphism(f_vec, A, alg, Ix=Ix)
    for v, r in frame_gen(x, alg, Ix=f_(Ix)):
        yield (r, lambda F: differential(F, f(x), v, h))

# Say f: M -> N
# "back" as the derivative acts on a field G on N
# Yet mimicking the derivative of G(f(x)) on M
def back_derivative_gen(x, f, alg, Ix, h=1e-6):
    f_vec = lambda a: lambda x: differential(f, x, a)
    f_ = lambda A: outermorphism(f_vec, A, alg, Ix=Ix)
    fa_ = lambda A: adjoint_outermorphism_(f_vec, A, alg, Ix=Ix)
    for v, r in frame_gen(x, alg, Ix=f_(Ix)):
        yield (fa_(r)(x), lambda F: differential(F, f(x), v, h))


fa_ = lambda A: adjoint_outermorphism_(f_vec, A, alg, Ix=Ix)
differential(F, x, pa(x)), sum((pa(x)|r) * o(G) for r, o in back_derivative_gen(x, f, alg, Ix))

(2.33 + -0.0953 𝐞₁ + -0.925 𝐞₂ + 0.495 𝐞₃ + 0.247 𝐞₁₂ + 0.463 𝐞₁₃ + -0.0476 𝐞₂₃ + 0.599 𝐞₁₂₃,
 2.33 + -0.0953 𝐞₁ + -0.925 𝐞₂ + 0.495 𝐞₃ + 0.248 𝐞₁₂ + 0.463 𝐞₁₃ + -0.0476 𝐞₂₃ + 0.599 𝐞₁₂₃)

In [13]:
#5.13
(
    derivative(F, x, alg, Ix=Ix), 
    sum(r * o(G) for r, o in back_derivative_gen(x, f, alg, Ix))
    )

(5.67 + 7.96 𝐞₁ + -12.2 𝐞₂ + 0.426 𝐞₃ + 0.121 𝐞₁₂ + 2.02 𝐞₁₃ + 1.81 𝐞₂₃ + 2.84 𝐞₁₂₃,
 5.67 + 7.96 𝐞₁ + -12.2 𝐞₂ + 0.426 𝐞₃ + 0.121 𝐞₁₂ + 2.02 𝐞₁₃ + 1.81 𝐞₂₃ + 2.84 𝐞₁₂₃)

In [14]:
def outermorphism_(f, A, alg, h=1e-6, frame=None, r_frame=None, Ix=None):
    def f_outer(x):
        nonlocal frame, r_frame
        if Ix:
            frame = blade_split(Ix(x), alg)
            r_frame = reciprocal(frame)
        wf = lambda vectors: wedge([f(v)(x) for v in vectors])
        outer = 0
        for r in A.grades:
            if r == 0:
                outer += A.grade(0)
                continue
            outer += skew_symmetrizer(wf, A.grade(r), alg, h, frame, r_frame)
        return outer
    return f_outer

In [16]:
#5.14 "forward" as the derivative acts on a field F on M
# Yet mimicking the derivative of F(inv(f)(x)) on N
def forward_derivative_gen(x, f, alg, Ix, h=1e-6):
    f_vec = lambda a: lambda x: differential(f, x, a)
    f__ = lambda A: outermorphism_(f_vec, A, alg, Ix=Ix)
    nIx = lambda x: normalize(Ix(x))
    nI = nIx(x)
    iI1 = inv(f_(nIx)(x))
    for v, r in frame_gen(x, alg, Ix=Ix):
        yield iI1 * f__(nI*r)(x), lambda F: differential(F, x, v, h)

iI1 = inv(f_(nIx)(x))
f__ = lambda A: outermorphism_(f_vec, A, alg, Ix=Ix)
(
    sum(r * o(G) for r, o in f_derivative_gen(x, f, alg, Ix)),
    sum(r * o(F) for r, o in forward_derivative_gen(x, f, alg, Ix)),
    sum(iI1 * f__(nI*r)(x) * o(F) for r, o in derivative_gen(x, alg, Ix=Ix))
    )

(1.95 + 1.97 𝐞₁ + -2.76 𝐞₂ + -0.198 𝐞₃ + -0.15 𝐞₁₂ + 0.782 𝐞₁₃ + 0.685 𝐞₂₃ + 0.975 𝐞₁₂₃,
 1.95 + 1.97 𝐞₁ + -2.76 𝐞₂ + -0.198 𝐞₃ + -0.15 𝐞₁₂ + 0.782 𝐞₁₃ + 0.685 𝐞₂₃ + 0.975 𝐞₁₂₃,
 1.95 + 1.97 𝐞₁ + -2.76 𝐞₂ + -0.198 𝐞₃ + -0.15 𝐞₁₂ + 0.782 𝐞₁₃ + 0.685 𝐞₂₃ + 0.975 𝐞₁₂₃)

In [17]:
#5.15a Without the inverse of f
# Pushing forward a differential is an extra linear transformation
b = random_r_blade(1, alg)
pb = P(b, I)
pb1 = f__(pb)(x)
differential(f__(PA(x)), x, pb, h=1e-2), sum((pb1 | r) * o(f__(PA(x))) for r, o in forward_derivative_gen(x, f, alg, Ix, h=1e-2))

(-0.124 𝐞₁ + 1.33 𝐞₂ + -0.17 𝐞₃ + 0.959 𝐞₁₂ + -0.202 𝐞₁₃ + 0.161 𝐞₂₃ + -7.81e-16 𝐞₁₂₃,
 5.7e-17 + -0.123 𝐞₁ + 1.32 𝐞₂ + -0.175 𝐞₃ + 0.953 𝐞₁₂ + -0.206 𝐞₁₃ + 0.163 𝐞₂₃ + -7.86e-16 𝐞₁₂₃)

In [18]:
#5.15b
differential(fa_(PA(x)), x, pb, h=1e-2), sum((pb1 | r) * o(fa_(PA(x))) for r, o in forward_derivative_gen(x, f, alg, Ix, h=1e-2))

(0.427 𝐞₁ + 1.02 𝐞₂ + -0.0449 𝐞₃ + 0.959 𝐞₁₂ + -0.0438 𝐞₁₃ + 0.0105 𝐞₂₃,
 1.81e-17 + 0.422 𝐞₁ + 1.02 𝐞₂ + -0.0448 𝐞₃ + 0.954 𝐞₁₂ + -0.0436 𝐞₁₃ + 0.0106 𝐞₂₃ + 8.99e-17 𝐞₁₂₃)

In [19]:
#5.16
(
    differential(f__(C^D), x, pb, h=1e-3), 
    (differential(f__(C), x, pb, h=1e-3)^f__(D)(x)) + (f__(C)(x)^differential(f__(D), x, pb, h=1e-3)),
    differential(fa_(C^D), x, pb, h=1e-3), 
    (differential(fa_(C), x, pb, h=1e-3)^fa_(D)(x)) + (fa_(C)(x)^differential(fa_(D), x, pb, h=1e-3)),
    )

(0.513 𝐞₁ + -4.88 𝐞₂ + 0.712 𝐞₃ + -8.94 𝐞₁₂ + 1.75 𝐞₁₃ + -1.46 𝐞₂₃ + -2.26e-14 𝐞₁₂₃,
 0.513 𝐞₁ + -4.88 𝐞₂ + 0.712 𝐞₃ + -8.94 𝐞₁₂ + 1.75 𝐞₁₃ + -1.46 𝐞₂₃ + -0.000276 𝐞₁₂₃,
 -1.94 𝐞₁ + -3.75 𝐞₂ + 0.111 𝐞₃ + -9.56 𝐞₁₂ + 0.524 𝐞₁₃ + -0.227 𝐞₂₃,
 -1.94 𝐞₁ + -3.75 𝐞₂ + 0.111 𝐞₃ + -9.56 𝐞₁₂ + 0.524 𝐞₁₃ + -0.227 𝐞₂₃ + 9.45e-06 𝐞₁₂₃)

In [20]:
#5.17a
Pb_ = lambda A: lambda x: differential(lambda a: P(A(x), Ix(a)), x, pb, h=1e-2)
Pb_1 = lambda A: lambda x: differential(lambda a: P(A(x), f_(Ix)(a)), x, pb, h=1e-2)
fb = lambda C: differential(f__(C), x, pb, h=1e-2)
C_ = lambda x: C
(
    fb(C),
    Pb_1(f__(C))(x)+ P(fb(C), I1),
    fb(P(C, I)) + f_(Pb_(C_))(x)
    )

(0.339 𝐞₁ + -3.55 𝐞₂ + 0.364 𝐞₃ + -7.49 𝐞₁₂ + 1.61 𝐞₁₃ + -1.33 𝐞₂₃ + 2.27e-15 𝐞₁₂₃,
 0.34 𝐞₁ + -3.55 𝐞₂ + 0.364 𝐞₃ + -7.48 𝐞₁₂ + 1.61 𝐞₁₃ + -1.33 𝐞₂₃,
 -1.11e-14 + 0.339 𝐞₁ + -3.55 𝐞₂ + 0.364 𝐞₃ + -7.49 𝐞₁₂ + 1.61 𝐞₁₃ + -1.33 𝐞₂₃ + 2.27e-15 𝐞₁₂₃)

In [21]:
#5.17b
fb = lambda C: differential(fa_(C), x, pb, h=1e-2)

(
    fb(C),
    Pb_(fa_(C))(x)+ P(fb(C), I),
    fb(P(C, I1)) + fa_(Pb_1(C_)(x))(x)
    )

(-1.27 𝐞₁ + -2.42 𝐞₂ + 0.0967 𝐞₃ + -7.98 𝐞₁₂ + 0.427 𝐞₁₃ + -0.175 𝐞₂₃,
 -1.11e-14 + -1.27 𝐞₁ + -2.42 𝐞₂ + 0.0966 𝐞₃ + -7.98 𝐞₁₂ + 0.427 𝐞₁₃ + -0.175 𝐞₂₃,
 -1.27 𝐞₁ + -2.42 𝐞₂ + 0.0966 𝐞₃ + -7.98 𝐞₁₂ + 0.427 𝐞₁₃ + -0.175 𝐞₂₃)

In [22]:
#5.18
differential(lambda y: differential(f, y, a(x)), x, b), differential(lambda y: differential(f, y, b), x, a(x))

(0.344 𝐞₁ + 3.28 𝐞₂ + -0.265 𝐞₃, 0.344 𝐞₁ + 3.28 𝐞₂ + -0.265 𝐞₃)

In [23]:
#5.19
tangents = blade_split(I, alg)
frame = multi_frame(tangents)
r_frame = reci_frame(tangents)
# pb_frame = [differential(lambda x: P(v, Ix(x)), x, b) for v in frame]
A1 = f_(PA)(x)
fb = lambda C: differential(fa_(C), x, pb, h=1e-2)
fb_A1 = lambda C: differential(f__(C), x, pb, h=1e-2).sp(A1)

(
    derivative(fb_A1, x, alg, frame=frame, r_frame=r_frame, h=1e-2) +
    Pb_(fa_(A1))(x),
    fb(A1)
    )

(2.78e-15 + 0.935 𝐞₁ + 3.62 𝐞₂ + -0.161 𝐞₃ + 8.29 𝐞₁₂ + -0.459 𝐞₁₃ + 0.203 𝐞₂₃ + 8.99e-18 𝐞₁₂₃,
 0.935 𝐞₁ + 3.62 𝐞₂ + -0.161 𝐞₃ + 8.29 𝐞₁₂ + -0.459 𝐞₁₃ + 0.203 𝐞₂₃)

In [24]:
#5.20
(
    derivative(fb_A1, x, alg, frame=frame, r_frame=r_frame, h=1e-2),
    P(Pb_(fa_(A1))(x), I),
    P(fb(A1), I)
    )

(0.933 𝐞₁ + 3.62 𝐞₂ + -0.069 𝐞₃ + 8.29 𝐞₁₂ + -0.199 𝐞₁₃ + -0.16 𝐞₂₃ + 8.99e-18 𝐞₁₂₃,
 2.78e-15 + 1.39e-07 𝐞₁ + 4.27e-07 𝐞₂ + -7.57e-09 𝐞₃ + -1.58e-07 𝐞₁₂ + 3.79e-09 𝐞₁₃ + 3.05e-09 𝐞₂₃,
 0.933 𝐞₁ + 3.62 𝐞₂ + -0.069 𝐞₃ + 8.29 𝐞₁₂ + -0.199 𝐞₁₃ + -0.16 𝐞₂₃)

In [25]:
#5.21
C.sp(fb(A1)), fb_A1(C)

(-3.83, -3.83)

In [26]:
#5.22 quadriple differential, so the truncations...
f_vec = lambda a: lambda x: differential(f, x, a, h=1e-3)
fa_ = lambda A: adjoint_outermorphism_(f_vec, A, alg, Ix=Ix, h=1e-3)
fb = lambda b: differential(fa_(A1), x, b, h=1e-3)
P(curl(fb, x, alg, Ix=Ix, h=1e-2), I)

-5.55e-20 𝐞₃ + 6.11e-05 𝐞₁₂ + -1.47e-06 𝐞₁₃ + -1.18e-06 𝐞₂₃

In [27]:
# Even though these two are the same
f__(PA(x))(x), f__(A(x))(x)

(-0.434 + -0.0976 𝐞₁ + -0.949 𝐞₂ + 0.507 𝐞₃ + -0.798 𝐞₁₂ + 0.366 𝐞₁₃ + -0.593 𝐞₂₃ + -4.62e-18 𝐞₁₂₃,
 -0.434 + -0.0976 𝐞₁ + -0.949 𝐞₂ + 0.507 𝐞₃ + -0.798 𝐞₁₂ + 0.366 𝐞₁₃ + -0.593 𝐞₂₃ + -4.62e-18 𝐞₁₂₃)

In [28]:
#5.23 The non-tangent part of A contributes to the exterior derivative

f__ = lambda A: outermorphism_(f_vec, A, alg, Ix=Ix, h=1e-3)
d(lambda x, A: f__(A)(x), x, A(x), alg, Ix, h=1e-2), d(lambda x, A: f__(A)(x), x, PA(x), alg, Ix, h=1e-2)

(-0.122 𝐞₁ + 0.102 𝐞₂ + 0.0441 𝐞₃ + -0.263 𝐞₁₂ + 0.121 𝐞₁₃ + -0.196 𝐞₂₃ + -8.77e-16 𝐞₁₂₃,
 -1.39e-06 𝐞₁ + -2.77e-06 𝐞₂ + -3.67e-06 𝐞₃ + 2.31e-19 𝐞₁₂ + -1.06e-19 𝐞₁₃ + 1.71e-19 𝐞₂₃ + -8.76e-16 𝐞₁₂₃)

In [29]:
#5.23 How? The vector part contributes nothing

f__ = lambda A: outermorphism_(f_vec, A, alg, Ix=Ix, h=1e-3)
d(lambda x, A: f__(A)(x), x, a(x), alg, Ix, h=1e-2), d(lambda x, A: f__(A)(x), x, pa(x), alg, Ix, h=1e-2)

(1.33e-17 𝐞₁₂ + -5.86e-18 𝐞₁₃ + 7.67e-18 𝐞₂₃,
 2.31e-19 𝐞₁₂ + -1.06e-19 𝐞₁₃ + 1.71e-19 𝐞₂₃)

In [30]:
#5.23 The bivector part contributes
blade = random_r_blade(2, alg)
pblade = P(blade, I)

f__ = lambda A: outermorphism_(f_vec, A, alg, Ix=Ix, h=1e-3)
d(lambda x, A: f__(A)(x), x, blade, alg, Ix, h=1e-2), d(lambda x, A: f__(A)(x), x, pblade, alg, Ix, h=1e-2)

(-0.00415 𝐞₁ + 0.0485 𝐞₂ + -0.0192 𝐞₃ + 4.43e-16 𝐞₁₂₃,
 7.04e-07 𝐞₁ + 1.4e-06 𝐞₂ + 1.86e-06 𝐞₃ + 4.43e-16 𝐞₁₂₃)

In [31]:
T = lambda x, A: f__(A)(x)
list((inner(blade, r), T(x, inner(blade, r)), o(lambda x: T(x, inner(blade, r)))) for r, o in derivative_gen(x, alg, Ix=nIx, h=1e-3))


[(-0.0498 𝐞₁ + -0.00896 𝐞₂ + -0.373 𝐞₃,
  -0.131 𝐞₁ + 0.0577 𝐞₂ + 0.0712 𝐞₃ + -3.77e-22 𝐞₁₂₃,
  -0.122 𝐞₁ + 0.0743 𝐞₂ + 0.0065 𝐞₃ + -8.08e-19 𝐞₁₂₃),
 (-0.00154 𝐞₁ + -0.0551 𝐞₂ + 0.08 𝐞₃,
  0.0573 𝐞₁ + -0.2 𝐞₂ + 0.0493 𝐞₃ + -8.16e-19 𝐞₁₂₃,
  0.118 𝐞₁ + -0.0258 𝐞₂ + -0.0257 𝐞₃ + -4.09e-16 𝐞₁₂₃)]

In [32]:
T = lambda x, A: f__(A)(x)
list(o(lambda x: T(x, inner(pblade, r))) for r, o in derivative_gen(x, alg, Ix=nIx, h=1e-3))


[-0.107 𝐞₁ + 0.0206 𝐞₂ + 0.0195 𝐞₃ + -9.13e-19 𝐞₁₂₃,
 0.107 𝐞₁ + -0.0206 𝐞₂ + -0.0195 𝐞₃ + -4.08e-16 𝐞₁₂₃]

In [33]:
#5.24
d(lambda x, A: P(A, Ix(x)), x, PA(x), alg, Ix)

-1.39e-11 + 1.4e-12 𝐞₁ + 6.93e-12 𝐞₂ + -4.77e-12 𝐞₃ + 3.26e-20 𝐞₁₂ + -7.81e-22 𝐞₁₃ + -6.27e-22 𝐞₂₃

In [34]:
#5.25
fb = lambda C: differential(fa_(C), x, pb, h=1e-3)
Pb_ = lambda A: lambda x: differential(lambda a: P(A(x), Ix(a)), x, pb, h=1e-3)
Pb_1 = lambda A: lambda x: differential(lambda a: P(A(x), f_(Ix)(a)), x, pb, h=1e-3)
P1C = P(C, I1)
C_ = lambda x: C - P1C
(
    fb(C),
    Pb_(fa_(C))(x)+ P(fb(C), I),
    fb(P1C) + fa_(Pb_1(C_)(x))(x)
    )

(-1.26 𝐞₁ + -2.42 𝐞₂ + 0.0969 𝐞₃ + -7.96 𝐞₁₂ + 0.427 𝐞₁₃ + -0.176 𝐞₂₃,
 -1.26 𝐞₁ + -2.42 𝐞₂ + 0.0969 𝐞₃ + -7.96 𝐞₁₂ + 0.427 𝐞₁₃ + -0.176 𝐞₂₃,
 -1.26 𝐞₁ + -2.42 𝐞₂ + 0.0969 𝐞₃ + -7.96 𝐞₁₂ + 0.427 𝐞₁₃ + -0.176 𝐞₂₃)

In [35]:
#5.25b
fb = lambda b: differential(fa_(C), x, b, h=1e-3)
Pb = lambda b: differential(lambda a: P(fa_(C)(x), Ix(a)), x, b, h=1e-3)
(
    curl(fb, x, alg, Ix=Ix, h=1e-3), 
    curl(Pb, x, alg, Ix=Ix, h=1e-3), 
    P(curl(fb, x, alg, Ix=Ix, h=1e-3), I)
    )


(0.00194 𝐞₁₂ + 0.00828 𝐞₁₃ + 0.0902 𝐞₂₃ + -0.944 𝐞₁₂₃,
 5.55e-11 𝐞₁ + 1.55e-10 𝐞₂ + -2.66e-12 𝐞₃ + 0.00194 𝐞₁₂ + 0.00829 𝐞₁₃ + 0.0903 𝐞₂₃ + -0.944 𝐞₁₂₃,
 -5.55e-20 𝐞₃ + 3.95e-06 𝐞₁₂ + -9.49e-08 𝐞₁₃ + -7.61e-08 𝐞₂₃)

In [36]:
#5.25c
fb = lambda b: differential(fa_(P1C), x, b, h=1e-3)
Pb_1 = lambda b: differential(lambda a: P(C_(x), f_(Ix)(a)), x, b, h=1e-3)
fa_ = lambda A: adjoint_outermorphism_(f_vec, A, alg, Ix=Ix, h=1e-3)
(
    curl(fb, x, alg, Ix=Ix, h=1e-3), 
    fa_(sum(r^o(Pb_1) for r, o in forward_derivative_gen(x, f, alg, Ix, h=1e-3)))(x),
    )


(0.00194 𝐞₁₂ + 0.00828 𝐞₁₃ + 0.0902 𝐞₂₃ + -0.943 𝐞₁₂₃,
 3.04e-26 𝐞₁ + 6.59e-23 𝐞₂ + -1.58e-24 𝐞₃ + -0.379 𝐞₁₂ + 0.00908 𝐞₁₃ + 0.00729 𝐞₂₃)

In [37]:
#5.26
S = lambda A: curl(lambda x: P(A, Ix(x)), x, alg, Ix=Ix, h=1e-3)
Pb = lambda b: differential(lambda a: P(fa_(C)(x), Ix(a)), x, b, h=1e-3)
fb = lambda C: lambda b: differential(fa_(C), x, b, h=1e-3)
(
    S(fa_(C)(x)), 
    curl(lambda a: P(fa_(C)(x), Ix(a)), x, alg, Ix=Ix, h=1e-3), 
    curl(Pb, x, alg, Ix=Ix, h=1e-3), 
    curl(fb(P1C), x, alg, Ix=Ix, h=1e-3), 
    curl(fb(C), x, alg, Ix=Ix, h=1e-3),
    )

(-1.67e-13 𝐞₁ + -1.55e-13 𝐞₂ + 5.18e-16 𝐞₃ + 0.00194 𝐞₁₂ + 0.00853 𝐞₁₃ + 0.0901 𝐞₂₃ + -0.943 𝐞₁₂₃,
 -1.67e-13 𝐞₁ + -1.55e-13 𝐞₂ + 5.18e-16 𝐞₃ + 0.00194 𝐞₁₂ + 0.00853 𝐞₁₃ + 0.0901 𝐞₂₃ + -0.943 𝐞₁₂₃,
 5.55e-11 𝐞₁ + 1.55e-10 𝐞₂ + -2.66e-12 𝐞₃ + 0.00194 𝐞₁₂ + 0.00829 𝐞₁₃ + 0.0903 𝐞₂₃ + -0.944 𝐞₁₂₃,
 0.00194 𝐞₁₂ + 0.00828 𝐞₁₃ + 0.0902 𝐞₂₃ + -0.943 𝐞₁₂₃,
 0.00194 𝐞₁₂ + 0.00828 𝐞₁₃ + 0.0902 𝐞₂₃ + -0.944 𝐞₁₂₃)

In [38]:
#5.27
C - P1C, curl(fb(C - P1C), x, alg, Ix=Ix, h=1e-3)

(1.11e-16 + 0.428 𝐞₁ + 0.264 𝐞₂ + 0.576 𝐞₃ + 0.164 𝐞₁₂ + 0.678 𝐞₁₃ + 0.198 𝐞₂₃ + 0.474 𝐞₁₂₃,
 -1.98e-06 𝐞₁₂ + -2.32e-06 𝐞₁₃ + 3.66e-05 𝐞₂₃ + -7.18e-05 𝐞₁₂₃)

In [39]:
#5.29 f: M -> N
# Let A be a field on M, also a field on N with a hidden inverse f

def forward_curl(F, x, f, alg, Ix, h=1e-6):
    return sum(r ^ o(F) for r, o in forward_derivative_gen(x, f, alg, Ix, h))

(
    curl(fa(A), x, alg, Ix=Ix, h=1e-3),
    S(fa(A)(x)) +
    fa_(forward_curl(A, x, f, alg, Ix, h=1e-3))(x)
    )

(-0.373 𝐞₁ + 0.613 𝐞₂ + -0.0219 𝐞₃ + 5.77 𝐞₁₂ + -0.118 𝐞₁₃ + -0.14 𝐞₂₃ + 0.528 𝐞₁₂₃,
 -0.373 𝐞₁ + 0.613 𝐞₂ + -0.0219 𝐞₃ + 5.77 𝐞₁₂ + -0.118 𝐞₁₃ + -0.14 𝐞₂₃ + 0.528 𝐞₁₂₃)

In [40]:
#5.30 cocurl "commutes" with adjoint with a pushforward
fA = fa(A)
(
    P(curl(fa(A), x, alg, Ix=Ix, h=1e-3), I), 
    fa_(forward_curl(A, x, f, alg, Ix, h=1e-3))(x), 
    sum(fa_(r)(x) ^ P(o(fA), I) for r, o in forward_derivative_gen(x, f, alg, Ix, h=1e-3)),
    sum(fa_(r)(x) ^ o(fA) for r, o in forward_derivative_gen(x, f, alg, Ix, h=1e-3)), # Wrong! Need a projection to keep it intrinsic
    )

(-0.373 𝐞₁ + 0.613 𝐞₂ + -0.0219 𝐞₃ + 5.77 𝐞₁₂ + -0.138 𝐞₁₃ + -0.111 𝐞₂₃,
 -0.373 𝐞₁ + 0.613 𝐞₂ + -0.0219 𝐞₃ + 5.77 𝐞₁₂ + -0.138 𝐞₁₃ + -0.111 𝐞₂₃,
 -0.373 𝐞₁ + 0.613 𝐞₂ + -0.0219 𝐞₃ + 5.77 𝐞₁₂ + -0.138 𝐞₁₃ + -0.111 𝐞₂₃,
 -0.373 𝐞₁ + 0.613 𝐞₂ + -0.0219 𝐞₃ + 5.77 𝐞₁₂ + -0.118 𝐞₁₃ + -0.14 𝐞₂₃ + 0.528 𝐞₁₂₃)

In [41]:
#5.31
nI1 = normalize(I1)
Jf = lambda x: norm(f_(nIx)(x))
fa_(f_(A)(x) * nI1)(x), Jf(x) * PA(x) * nI

(1.06 + 3.03 𝐞₁ + -1.72 𝐞₂ + 0.0997 𝐞₃ + -4.08 𝐞₁₂ + 0.0978 𝐞₁₃ + 0.0785 𝐞₂₃,
 1.06 + 3.03 𝐞₁ + -1.72 𝐞₂ + 0.0997 𝐞₃ + -4.08 𝐞₁₂ + 0.0978 𝐞₁₃ + 0.0785 𝐞₂₃ + -6.94e-18 𝐞₁₂₃)

In [42]:
#5.32a
codiv_ = lambda A: codiv(A, x, alg, Ix=Ix, h=1e-3)
cocurl_ = lambda A: cocurl(A, x, alg, Ix=Ix, h=1e-3)
(
    sum(inner(r, o(f_(PA))) for r, o in forward_derivative_gen(x, f, alg, Ix, h=1e-2)),
    f__(codiv_(lambda x: Jf(x)*PA(x))/Jf(x))(x),
    f__(codiv_(PA) + inner(cocurl_(lambda x: np.log(Jf(x))),PA(x)))(x)
    )

(0.786 + -2.55 𝐞₁ + 6.02 𝐞₂ + -0.869 𝐞₃ + 3.01e-05 𝐞₁₂ + 4.19e-05 𝐞₁₃ + -1.52e-05 𝐞₂₃,
 0.786 + -2.55 𝐞₁ + 6.02 𝐞₂ + -0.868 𝐞₃ + -2.31e-19 𝐞₁₂ + 1.06e-19 𝐞₁₃ + -1.71e-19 𝐞₂₃ + 2.28e-17 𝐞₁₂₃,
 0.786 + -2.55 𝐞₁ + 6.02 𝐞₂ + -0.868 𝐞₃ + -2.31e-19 𝐞₁₂ + 1.06e-19 𝐞₁₃ + -1.71e-19 𝐞₂₃ + 2.28e-17 𝐞₁₂₃)

In [43]:
#5.32b
div_ = lambda A: div(A, x, alg, Ix=Ix, h=1e-3)
curl_ = lambda A: curl(A, x, alg, Ix=Ix, h=1e-3)
(
    sum(inner(r, o(f_(PA))) for r, o in forward_derivative_gen(x, f, alg, Ix, h=1e-2)),
    f__(div_(lambda x: Jf(x)*PA(x))/Jf(x))(x),
    f__(div_(PA) + inner(curl_(lambda x: np.log(Jf(x))),PA(x)))(x)
    )

(0.786 + -2.55 𝐞₁ + 6.02 𝐞₂ + -0.869 𝐞₃ + 3.01e-05 𝐞₁₂ + 4.19e-05 𝐞₁₃ + -1.52e-05 𝐞₂₃,
 0.786 + -2.55 𝐞₁ + 6.02 𝐞₂ + -0.868 𝐞₃ + -2.31e-19 𝐞₁₂ + 1.06e-19 𝐞₁₃ + -1.71e-19 𝐞₂₃ + 2.28e-17 𝐞₁₂₃,
 0.786 + -2.55 𝐞₁ + 6.02 𝐞₂ + -0.868 𝐞₃ + -2.31e-19 𝐞₁₂ + 1.06e-19 𝐞₁₃ + -1.71e-19 𝐞₂₃ + 2.28e-17 𝐞₁₂₃)

In [44]:
#5.33
inner(b, f_(A)(x)), f__(inner(fa_(b)(x), A(x)))(x)

(-0.512 + 0.404 𝐞₁ + -0.268 𝐞₂ + -0.177 𝐞₃ + -1.47e-18 𝐞₁₂ + 3.01e-18 𝐞₁₃ + -2.64e-18 𝐞₂₃,
 -0.512 + 0.404 𝐞₁ + -0.268 𝐞₂ + -0.177 𝐞₃ + -1.07e-17 𝐞₁₂ + 4.93e-18 𝐞₁₃ + -7.98e-18 𝐞₂₃ + -4.22e-19 𝐞₁₂₃)

In [45]:
#5.34 codiv "commutes" with outermorphism also with a pushforward but on the LHS
(
    sum(inner(r, f__(o(A))(x)) for r, o in forward_derivative_gen(x, f, alg, Ix)),
    f__(codiv_(A))(x)
    )

(0.296 + -2.0 𝐞₁ + 6.05 𝐞₂ + -1.29 𝐞₃ + 2.33e-05 𝐞₁₂ + 1.71e-05 𝐞₁₃ + -2.09e-05 𝐞₂₃,
 0.296 + -2.0 𝐞₁ + 6.05 𝐞₂ + -1.29 𝐞₃ + 3.52e-17 𝐞₁₂ + -1.61e-17 𝐞₁₃ + 2.61e-17 𝐞₂₃ + 2.41e-17 𝐞₁₂₃)

In [46]:
#5.35
def forward_curl(F, x, f, alg, Ix, h=1e-3):
    return sum(r ^ o(F) for r, o in forward_derivative_gen(x, f, alg, Ix, h=h))

def forward_div(F, x, f, alg, Ix, h=1e-3):
    return sum(inner(r, o(F)) for r, o in forward_derivative_gen(x, f, alg, Ix, h=1e-3))
(
    forward_div(f__(PA(x)), x, f, alg, Ix),
    f__(inner(curl_(lambda x: np.log(Jf(x))), A(x)))(x),
    inner(forward_curl(lambda x: np.log(Jf(x)), x, f, alg, Ix), f_(A)(x)),
)

(0.523 + -0.455 𝐞₁ + -0.145 𝐞₂ + 0.405 𝐞₃ + 6.87e-06 𝐞₁₂ + 2.56e-05 𝐞₁₃ + 5.93e-06 𝐞₂₃,
 0.523 + -0.455 𝐞₁ + -0.145 𝐞₂ + 0.405 𝐞₃ + -2.58e-17 𝐞₁₂ + 1.18e-17 𝐞₁₃ + -1.92e-17 𝐞₂₃ + -1.61e-18 𝐞₁₂₃,
 0.523 + -0.455 𝐞₁ + -0.145 𝐞₂ + 0.405 𝐞₃ + 1.77e-05 𝐞₁₂ + 3.3e-05 𝐞₁₃ + -3.4e-06 𝐞₂₃)

In [47]:
#5.36 simplify to differential only when A is a vector
(   
    differential(G, f(x), f_(A)(x), h=1e-3), 
    differential(G, f(x), f_(a)(x), h=1e-3), 
    sum(f__(alg.sp(r, a(x)))(x) * o(F) for r, o in derivative_gen(x, alg, Ix=Ix, h=1e-3)),
    sum(f__(inner(A(x), r))(x) * o(F) for r, o in derivative_gen(x, alg, Ix=Ix, h=1e-3))
    )

(2.92 + -0.282 𝐞₁ + 0.687 𝐞₂ + 1.48 𝐞₃ + -0.779 𝐞₁₂ + 0.624 𝐞₁₃ + -1.16 𝐞₂₃ + 2.04 𝐞₁₂₃,
 2.33 + -0.0953 𝐞₁ + -0.925 𝐞₂ + 0.495 𝐞₃ + 0.247 𝐞₁₂ + 0.463 𝐞₁₃ + -0.0476 𝐞₂₃ + 0.599 𝐞₁₂₃,
 2.33 + -0.0953 𝐞₁ + -0.925 𝐞₂ + 0.495 𝐞₃ + 0.247 𝐞₁₂ + 0.463 𝐞₁₃ + -0.0476 𝐞₂₃ + 0.599 𝐞₁₂₃,
 2.33 + 2.62 𝐞₁ + 1.12 𝐞₂ + -1.09 𝐞₃ + -2.02 𝐞₁₂ + 0.541 𝐞₁₃ + -0.635 𝐞₂₃ + 0.599 𝐞₁₂₃)

In [48]:
#5.36
(
    sum(inner(f__(A(x))(x), r) * o(G) for r, o in f_derivative_gen(x, f, alg, Ix, h=1e-3)),
    sum(inner(f__(A(x))(x), r) * o(F) for r, o in forward_derivative_gen(x, f, alg, Ix, h=1e-3)),
    sum(f__(inner(A(x), r))(x) * o(F) for r, o in derivative_gen(x, alg, Ix=Ix, h=1e-3))
    )

(2.33 + 2.62 𝐞₁ + 1.12 𝐞₂ + -1.09 𝐞₃ + -2.02 𝐞₁₂ + 0.541 𝐞₁₃ + -0.635 𝐞₂₃ + 0.599 𝐞₁₂₃,
 2.33 + 2.62 𝐞₁ + 1.12 𝐞₂ + -1.09 𝐞₃ + -2.02 𝐞₁₂ + 0.541 𝐞₁₃ + -0.635 𝐞₂₃ + 0.599 𝐞₁₂₃,
 2.33 + 2.62 𝐞₁ + 1.12 𝐞₂ + -1.09 𝐞₃ + -2.02 𝐞₁₂ + 0.541 𝐞₁₃ + -0.635 𝐞₂₃ + 0.599 𝐞₁₂₃)

In [49]:
#5.37 holds for tangent fields
B = lambda x: (C*x+D)**2
def forward_lie_bracket(A, B, f, alg, Ix, h=1e-6):
    return lambda x: sum((inner(A(x),r)^o(B)) - (o(A)^inner(r,B(x))) for r, o in forward_derivative_gen(x, f, alg, Ix, h))

PB = lambda x: P(B(x), Ix(x))

(
    f_(lie_bracket(A, B, alg, Ix))(x), 
    forward_lie_bracket(f_(A), f_(B), f, alg, Ix, h=1e-3)(x), 
    f_(lie_bracket(PA, PB, alg, Ix))(x), 
    forward_lie_bracket(f_(PA), f_(PB), f, alg, Ix, h=1e-3)(x)
    )

(0.441 + 0.921 𝐞₁ + 0.256 𝐞₂ + -0.802 𝐞₃ + 3.04 𝐞₁₂ + -1.39 𝐞₁₃ + 2.26 𝐞₂₃ + 3.08e-18 𝐞₁₂₃,
 0.441 + 0.926 𝐞₁ + 0.271 𝐞₂ + -0.812 𝐞₃ + 3.1 𝐞₁₂ + -1.42 𝐞₁₃ + 2.3 𝐞₂₃ + 0.000112 𝐞₁₂₃,
 0.441 + 0.926 𝐞₁ + 0.271 𝐞₂ + -0.812 𝐞₃ + 3.1 𝐞₁₂ + -1.42 𝐞₁₃ + 2.3 𝐞₂₃ + 3.16e-18 𝐞₁₂₃,
 0.441 + 0.926 𝐞₁ + 0.271 𝐞₂ + -0.812 𝐞₃ + 3.1 𝐞₁₂ + -1.42 𝐞₁₃ + 2.3 𝐞₂₃ + 0.000112 𝐞₁₂₃)

In [50]:
#5.39
r = 1
s = 2
P1B = lambda x: P(B(x), f_(Ix)(x))
Ar = lambda x: PA(x).grade(r)
Bs = lambda x: P1B(x).grade(s)

def forward_dual_bracket(A, B, f, alg, nIx, h=1e-6):
    # nIx: normalized pseudoscalar
    f_vec = lambda a: lambda x: differential(f, x, a)
    f_ = lambda A: outermorphism(f_vec, A, alg, Ix=Ix)
    nIx = lambda x: normalize(Ix(x))
    return lambda x: forward_lie_bracket(A, lambda x: B(x)*f_(nIx)(x), f, alg, nIx, h)(x) * inv(f_(nIx)(x))
(
    fa(forward_dual_bracket(f_(Ar), Bs, f, alg, nIx, h=1e-3))(x),
    dual_bracket(Ar, fa(Bs), alg, nIx, h=1e-3)(x) -
    inner(inner(Ar(x), cocurl_(lambda x: np.log(Jf(x)))), fa(Bs)(x))
)

(1.16e-07 + 5.47 𝐞₁₂ + -0.131 𝐞₁₃ + -0.105 𝐞₂₃,
 1.49e-17 + 5.44 𝐞₁₂ + -0.131 𝐞₁₃ + -0.105 𝐞₂₃)

In [51]:
#6.1
s = lambda x: ((C + D*x) ** 2)[0]
f = lambda x: s(x)*x
s(x), terms_ratio(x, f(x)), terms_ratio(Ix(x), f_(Ix)(x))

(-0.2716784730617152,
 array([-3.68082163, -3.68082163, -3.68082163]),
 array([ 8.57262354, -0.22564944,  0.26457783]))

In [52]:
#6.2
r = 2
Ar = lambda x: A(x).grade(r)
PAr = P(Ar(x), I)
(
    f_(Ar)(x), 
    (s(x) ** r * PAr) + (s(x)**(r-1) * sum((PAr|r) * o(s) for r, o in derivative_gen(x, alg, Ix=Ix)) ^ x),
    s(x)** r * (PAr + (sum((PAr|r) * o(lambda x: np.log(abs(s(x)))) for r, o in derivative_gen(x, alg, Ix=Ix)) ^ x))
    )

(-0.0131 𝐞₁₂ + -0.012 𝐞₁₃ + 0.0082 𝐞₂₃,
 -0.0131 𝐞₁₂ + -0.012 𝐞₁₃ + 0.0082 𝐞₂₃,
 -0.0131 𝐞₁₂ + -0.012 𝐞₁₃ + 0.0082 𝐞₂₃)

In [53]:
#6.2 on Ix
(
    f_(Ix)(x), 
    (s(x) ** r * I) + (s(x)**(r-1) * sum((I|r) * o(s) for r, o in derivative_gen(x, alg, Ix=Ix)) ^ x),
    (s(x) ** r * I), (s(x)**(r-1) * sum((I|r) * o(s) for r, o in derivative_gen(x, alg, Ix=Ix)) ^ x),
    )

(0.326 𝐞₁₂ + 0.297 𝐞₁₃ + -0.203 𝐞₂₃,
 0.326 𝐞₁₂ + 0.297 𝐞₁₃ + -0.203 𝐞₂₃,
 0.206 𝐞₁₂ + -0.00495 𝐞₁₃ + -0.00397 𝐞₂₃,
 0.12 𝐞₁₂ + 0.302 𝐞₁₃ + -0.199 𝐞₂₃)

In [54]:
#6.3
(
    fa(Ar)(x), 
    (s(x) ** r * PAr) + (s(x)**(r-1) * derivative(s, x, alg, Ix=Ix) ^ P(x|Ar(x), I)),
    )

(0.0926 𝐞₁₂ + -0.00222 𝐞₁₃ + -0.00178 𝐞₂₃,
 0.0926 𝐞₁₂ + -0.00222 𝐞₁₃ + -0.00178 𝐞₂₃)

In [55]:
#6.4 central projection (normalization)
s = lambda x: 1/norm(x) # so f(x) = normalize(x)
I1x = lambda x: f(x) * e123
n = lambda x: -e123 * nIx(x)
nI, n(x) * e123, e123*n(x)

(1.0 𝐞₁₂ + -0.024 𝐞₁₃ + -0.0192 𝐞₂₃,
 1.0 𝐞₁₂ + -0.024 𝐞₁₃ + -0.0192 𝐞₂₃,
 1.0 𝐞₁₂ + -0.024 𝐞₁₃ + -0.0192 𝐞₂₃)

In [56]:
#6.5a The Jf in our example indicates an opposite orientation
f_(nIx)(x), Jf(x)*I1x(x), (s(x) ** 2 * nI ) + ((s(x)*nI*derivative(s, x, alg, Ix=Ix)) ^ x)

(0.492 𝐞₁₂ + -0.295 𝐞₁₃ + -0.152 𝐞₂₃,
 -0.492 𝐞₁₂ + 0.295 𝐞₁₃ + 0.152 𝐞₂₃,
 0.492 𝐞₁₂ + -0.295 𝐞₁₃ + -0.152 𝐞₂₃)

In [57]:
#6.5b
f_(nIx)(x)*e123.reverse(), Jf(x)*normalize(x), (s(x) ** 2 * n(x) ) + ((s(x)*n(x)^derivative(s, x, alg, Ix=Ix)) | x)

(-0.152 𝐞₁ + 0.295 𝐞₂ + 0.492 𝐞₃,
 0.152 𝐞₁ + -0.295 𝐞₂ + -0.492 𝐞₃,
 -0.152 𝐞₁ + 0.295 𝐞₂ + 0.492 𝐞₃)

In [58]:
#6.6 The f-inverse from S to M is just x -> x/s(x)
x_ = f(x)

n(x)/Jf(x), (s(x) ** (-2) * x_ ) + ((x_^forward_curl(lambda x: 1/s(x), x, f, alg, Ix=Ix)) | x_)/s(x)

(-0.0324 𝐞₁ + 0.0404 𝐞₂ + 1.69 𝐞₃, 0.0324 𝐞₁ + -0.0404 𝐞₂ + -1.69 𝐞₃)

In [59]:
#6.7 terms in the RHS of 6.6 are perpendicular, so square and sum
fc = forward_curl(lambda x: 1/s(x), x, f, alg, Ix=Ix)
Jf(x), 1/np.sqrt(((s(x) ** (-4)) + (s(x)**(-2)*fc**2))[0]), s(x) ** 2 * n(x) | x_

(0.5931313988825759, 0.5931269175909278, -0.593)

In [60]:
#6.9 inversion
s = lambda x: 1/(x**2)[0]
Ar = lambda x: PA(x).grade(r)
f_(Ar)(x), fa(Ar)(x) # not equal for all manifolds

(-0.0223 𝐞₁₂ + 0.0453 𝐞₁₃ + 0.0229 𝐞₂₃,
 -0.0238 𝐞₁₂ + 0.000572 𝐞₁₃ + 0.000459 𝐞₂₃)

In [61]:
# But on Euclidean space
Ix = lambda x: e123
f_(Ar)(x), fa(Ar)(x)

(-0.236 𝐞₁₂ + -0.186 𝐞₁₃ + 0.272 𝐞₂₃, -0.236 𝐞₁₂ + -0.186 𝐞₁₃ + 0.272 𝐞₂₃)

In [62]:
# inversion is curl free
curl(f, x, alg, grade=1)

1.39e-11 𝐞₁₂ + -1.39e-11 𝐞₁₃ + 5.55e-11 𝐞₂₃

In [63]:
#6.10
def forward_derivative(F, x, f, alg, Ix, h=1e-3):
    return sum(r * o(F) for r, o in forward_derivative_gen(x, f, alg, Ix, h=1e-3))

(
    derivative(F, x, alg, grade=1), 
    sum(-x.sw(r)/(x**4)[0] * o(F) for r, o in forward_derivative_gen(x, f, alg, Ix, h=1e-3)),
    forward_derivative(F, x, f, alg, Ix),
    sum(-x.sw(r) * o(F) for r, o in derivative_gen(x, alg, Ix=Ix, h=1e-3))
    )

(0.684 + 0.524 𝐞₁ + 0.771 𝐞₂ + -0.202 𝐞₃ + 0.474 𝐞₁₂ + 0.273 𝐞₁₃ + 0.834 𝐞₂₃ + 0.342 𝐞₁₂₃,
 0.684 + 0.524 𝐞₁ + 0.771 𝐞₂ + -0.202 𝐞₃ + 0.474 𝐞₁₂ + 0.273 𝐞₁₃ + 0.834 𝐞₂₃ + 0.342 𝐞₁₂₃,
 2.93 + 0.807 𝐞₁ + 0.983 𝐞₂ + -0.481 𝐞₃ + 0.571 𝐞₁₂ + 0.451 𝐞₁₃ + 1.22 𝐞₂₃ + 1.46 𝐞₁₂₃,
 2.93 + 0.806 𝐞₁ + 0.982 𝐞₂ + -0.481 𝐞₃ + 0.571 𝐞₁₂ + 0.451 𝐞₁₃ + 1.22 𝐞₂₃ + 1.46 𝐞₁₂₃)

In [64]:
#6.11 It's wrong because the derivative measures several nearby points around x, 
# but not directly on x, so no direct cancellation of x
deriv_ = lambda F: lambda x: derivative(F, x, alg, Ix=Ix, h=1e-3)
f_deriv_ = lambda F: lambda x: sum(r/(x**4)[0] * o(F) for r, o in forward_derivative_gen(x, f, alg, Ix, h=1e-3))
forward_deriv_ = lambda F: lambda x: sum(r * o(F) for r, o in forward_derivative_gen(x, f, alg, Ix, h=1e-3))
ff_deriv_ = lambda F: lambda x: sum(-x.sw(r)/(x**4)[0] * o(F) for r, o in forward_derivative_gen(x, f, alg, Ix, h=1e-3))

(
    deriv_(deriv_(F))(x), 
    f_deriv_(f_deriv_(F))(x), 
    forward_deriv_(forward_deriv_(F))(x)/(x**4)[0], 
    ff_deriv_(ff_deriv_(F))(x)
    )

(2.82 + -0.293 𝐞₁ + 0.57 𝐞₂ + 0.95 𝐞₃ + 0.475 𝐞₁₂ + -0.285 𝐞₁₃ + -0.146 𝐞₂₃ + -0.0736 𝐞₁₂₃,
 1.58 + 1.7 𝐞₁ + -0.401 𝐞₂ + -3.47 𝐞₃ + -0.637 𝐞₁₂ + 1.53 𝐞₁₃ + 1.71 𝐞₂₃ + 0.0958 𝐞₁₂₃,
 2.94 + 0.157 𝐞₁ + -0.117 𝐞₂ + -0.184 𝐞₃ + 0.0248 𝐞₁₂ + 0.111 𝐞₁₃ + 0.123 𝐞₂₃ + 0.0475 𝐞₁₂₃,
 2.8 + -0.336 𝐞₁ + 0.479 𝐞₂ + 1.04 𝐞₃ + 0.504 𝐞₁₂ + -0.351 𝐞₁₃ + -0.243 𝐞₂₃ + -0.0261 𝐞₁₂₃)

In [65]:
#6.12
Jf(x), -e123*f__(e123)(x), -1/(x**6)[0]

(0.34512491747611357, -0.345, -0.34512494149178446)

In [66]:
def Ix(x):
    p = np.arctan((4/5)*x.e2/x.e1)
    t = np.arccos(x.e3)
    return e_p(p, t) ^ e_t(p, t)

In [67]:
#6.14
n = normalize(random_r_blade(1, alg))
s = lambda x: ((C + D*x) ** 2)[0]
f = lambda x: x + n*s(x)
ds = derivative(s, x, alg, Ix=Ix)
f_(A)(x), PA(x) + (inner(PA(x),ds) ^ n)

(-0.434 + -0.204 𝐞₁ + -0.491 𝐞₂ + -0.0396 𝐞₃ + -0.159 𝐞₁₂ + -0.00831 𝐞₁₃ + 0.0109 𝐞₂₃ + -8.01e-19 𝐞₁₂₃,
 -0.434 + -0.204 𝐞₁ + -0.491 𝐞₂ + -0.0396 𝐞₃ + -0.159 𝐞₁₂ + -0.00831 𝐞₁₃ + 0.0109 𝐞₂₃ + -3.26e-19 𝐞₁₂₃)

In [68]:
#6.15
fa(A)(x), P(A(x) + (ds ^ inner(n, A(x))), I)

(-0.434 + -0.29 𝐞₁ + -0.488 𝐞₂ + 0.00612 𝐞₃ + -0.0549 𝐞₁₂ + 0.00132 𝐞₁₃ + 0.00106 𝐞₂₃,
 -0.434 + -0.29 𝐞₁ + -0.488 𝐞₂ + 0.00612 𝐞₃ + -0.0549 𝐞₁₂ + 0.00132 𝐞₁₃ + 0.00106 𝐞₂₃)

In [69]:
#6.16 because s is scalar, ds is tangent
Pn = P(n,I)
Kf = nI.reverse() * f__(nI)(x)
Kf, 1 + differential(s, x, Pn) + (ds^(n - Pn))


(1.41 + 0.00374 𝐞₁₂ + 0.0695 𝐞₁₃ + 0.108 𝐞₂₃,
 1.41 + 0.00374 𝐞₁₂ + 0.0695 𝐞₁₃ + 0.108 𝐞₂₃)

In [70]:
#6.17 Kf**2 = (Ir * f_(I))r * Ir * f_(I) = f_(I)r * f_(I) = Jf**2
normsq(Kf), Jf(x) ** 2, (1 + differential(s, x, Pn))**2 + ds**2 * normsq(nI ^n)

(2.009547859150122,
 2.009547793220037,
 2.01 + -6.62e-22 𝐞₁₂ + -6.04e-20 𝐞₁₃ + 3.44e-20 𝐞₂₃)

In [71]:
#6.18
(
    deriv_(F)(x), 
    sum(fa_(r)(x) * o(F) for r, o in forward_derivative_gen(x, f, alg, Ix, h=1e-3)),
    sum(P(r + ds * (n|r), I) * o(F) for r, o in forward_derivative_gen(x, f, alg, Ix, h=1e-3)),
    )

(2.35 + 0.826 𝐞₁ + 0.132 𝐞₂ + -0.0826 𝐞₃ + 0.23 𝐞₁₂ + 0.585 𝐞₁₃ + 1.3 𝐞₂₃ + 1.18 𝐞₁₂₃,
 2.35 + 0.826 𝐞₁ + 0.132 𝐞₂ + -0.0826 𝐞₃ + 0.23 𝐞₁₂ + 0.585 𝐞₁₃ + 1.3 𝐞₂₃ + 1.18 𝐞₁₂₃,
 2.35 + 0.826 𝐞₁ + 0.132 𝐞₂ + -0.0826 𝐞₃ + 0.23 𝐞₁₂ + 0.585 𝐞₁₃ + 1.3 𝐞₂₃ + 1.18 𝐞₁₂₃)

In [72]:
#6.19
Ix = lambda x: n * e123
I = Ix(x)
v = random_r_blade(1, alg)
x = v - P(v, n)
x|n, P(n, I)

(, )

In [73]:
#6.20b
I1 = f_(Ix)(x)
nI = nIx(x)
nI1 = normalize(I1)
n1 = -nI1 * e123
nI1, n1*e123

(0.893 𝐞₁₂ + -0.395 𝐞₁₃ + -0.217 𝐞₂₃, 0.893 𝐞₁₂ + -0.395 𝐞₁₃ + -0.217 𝐞₂₃)

In [74]:
#6.21
ds = derivative(s, x, alg, Ix=Ix)
f_(A)(x), PA(x) + (inner(PA(x),ds) ^ n), PA(x) + (inner(PA(x),ds) * n)

(0.27 + -0.243 𝐞₁ + 0.273 𝐞₂ + -0.18 𝐞₃ + 0.704 𝐞₁₂ + -0.311 𝐞₁₃ + -0.171 𝐞₂₃ + 8.12e-18 𝐞₁₂₃,
 0.27 + -0.243 𝐞₁ + 0.273 𝐞₂ + -0.18 𝐞₃ + 0.704 𝐞₁₂ + -0.311 𝐞₁₃ + -0.171 𝐞₂₃ + 1.44e-34 𝐞₁₂₃,
 0.27 + -0.243 𝐞₁ + 0.273 𝐞₂ + -0.18 𝐞₃ + 0.704 𝐞₁₂ + -0.311 𝐞₁₃ + -0.171 𝐞₂₃ + 1.44e-34 𝐞₁₂₃)

In [75]:
nIx(x), f_(nIx)(x), nI * (1 + (ds ^ n)), Jf(x)*nI1

(0.25 𝐞₁₂ + -0.961 𝐞₁₃ + 0.121 𝐞₂₃,
 1.55 𝐞₁₂ + -0.685 𝐞₁₃ + -0.377 𝐞₂₃,
 -5.55e-17 + 1.55 𝐞₁₂ + -0.685 𝐞₁₃ + -0.377 𝐞₂₃,
 1.55 𝐞₁₂ + -0.685 𝐞₁₃ + -0.377 𝐞₂₃)

In [76]:
#6.22
-nI * f_(nIx)(x), Jf(x) * n * n1, 1 + ds*n

(1.0 + 0.445 𝐞₁₂ + 0.281 𝐞₁₃ + 1.32 𝐞₂₃,
 1.0 + 0.445 𝐞₁₂ + 0.281 𝐞₁₃ + 1.32 𝐞₂₃,
 1.0 + 0.445 𝐞₁₂ + 0.281 𝐞₁₃ + 1.32 𝐞₂₃)

In [77]:
#6.23
n1, (n - ds)/Jf(x)

(-0.217 𝐞₁ + 0.395 𝐞₂ + 0.893 𝐞₃,
 -0.217 𝐞₁ + 0.395 𝐞₂ + 0.893 𝐞₃ + 5.46e-18 𝐞₁₂₃)

In [78]:
#6.24
Jf(x), norm(n - ds)

(1.735448691590876, 1.7354488877717207)

In [79]:
#6.25
derivative(F, x, alg, Ix=Ix), sum((P(r, I)+ds*(n|r))*o(F) for r, o in forward_derivative_gen(x, f, alg, Ix))

(1.95 + -0.0748 𝐞₁ + -0.0165 𝐞₂ + 0.0994 𝐞₃ + 1.23 𝐞₁₂ + 0.628 𝐞₁₃ + 2.45 𝐞₂₃ + 0.975 𝐞₁₂₃,
 1.95 + -0.0747 𝐞₁ + -0.0164 𝐞₂ + 0.0997 𝐞₃ + 1.23 𝐞₁₂ + 0.628 𝐞₁₃ + 2.45 𝐞₂₃ + 0.975 𝐞₁₂₃)

In [80]:
#6.26
s(x), n | f(x)

(-0.4670186621038733, -0.467)

In [81]:
#6.27
x1 = f(x)
f(P(x1, I)), x1

(-0.109 𝐞₁ + -0.614 𝐞₂ + 0.543 𝐞₃, -0.109 𝐞₁ + -0.614 𝐞₂ + 0.543 𝐞₃)

In [82]:
#6.28
g = lambda x: P(x, I)
g_vec = lambda a: lambda x: differential(g, x, a)
g_ = lambda A: outermorphism(g_vec, A, alg, Ix=f_(Ix))
P1A = lambda x: P(A(x), I1)
g_(P1A)(x), P(P1A(x), I)

(0.27 + -0.229 𝐞₁ + 0.112 𝐞₂ + -0.321 𝐞₃ + 0.107 𝐞₁₂ + -0.412 𝐞₁₃ + 0.0519 𝐞₂₃,
 0.27 + -0.229 𝐞₁ + 0.112 𝐞₂ + -0.321 𝐞₃ + 0.107 𝐞₁₂ + -0.412 𝐞₁₃ + 0.0519 𝐞₂₃)

In [83]:
#6.29
ga = lambda A: adjoint_outermorphism(g_vec, A, alg, Ix=f_(Ix))
ga(A)(x), P(PA(x), I1)

(0.27 + -0.289 𝐞₁ + 0.135 𝐞₂ + -0.13 𝐞₃ + 0.234 𝐞₁₂ + -0.103 𝐞₁₃ + -0.0569 𝐞₂₃,
 0.27 + -0.289 𝐞₁ + 0.135 𝐞₂ + -0.13 𝐞₃ + 0.234 𝐞₁₂ + -0.103 𝐞₁₃ + -0.0569 𝐞₂₃)

In [84]:
#6.30
forward_deriv_(F)(x), sum(P(r, I1)*o(F) for r, o in derivative_gen(x, alg, Ix=Ix))

(1.95 + 0.589 𝐞₁ + -0.193 𝐞₂ + 0.229 𝐞₃ + 0.512 𝐞₁₂ + 0.477 𝐞₁₃ + 1.23 𝐞₂₃ + 0.975 𝐞₁₂₃,
 1.95 + 0.589 𝐞₁ + -0.193 𝐞₂ + 0.228 𝐞₃ + 0.512 𝐞₁₂ + 0.478 𝐞₁₃ + 1.24 𝐞₂₃ + 0.975 𝐞₁₂₃)

In [85]:
#6.31
Ix = lambda x: n * e123
l = lambda x: 0.5 * normsq(x-n)
f = lambda x: n + ((x-n)/l(x))
f(x), -(x-n).sw(n)/normsq(x-n)

(-0.116 𝐞₁ + -0.576 𝐞₂ + 0.809 𝐞₃, -0.116 𝐞₁ + -0.576 𝐞₂ + 0.809 𝐞₃)

In [86]:
#6.34
f_(pa)(x), -(x-n).sw(pa(x))/normsq(x-n)/l(x)

(-0.411 𝐞₁ + -0.19 𝐞₂ + -0.195 𝐞₃ + 1.11e-17 𝐞₁₂₃,
 -0.411 𝐞₁ + -0.19 𝐞₂ + -0.195 𝐞₃)

In [87]:
#6.35
pb = P(b, I)
f_(pa)(x) * f__(pb)(x), (x-n).sw(pa(x)*pb)/normsq(x-n)/l(x)**2

(-0.305 + 0.119 𝐞₁₂ + 0.0844 𝐞₁₃ + -0.0171 𝐞₂₃,
 -0.305 + 0.119 𝐞₁₂ + 0.0844 𝐞₁₃ + -0.0171 𝐞₂₃)

In [88]:
#6.36
pa(x) | pb / l(x)**2

-0.305

In [89]:
#6.37
f__(pa(x) ^ pb)(x)

0.119 𝐞₁₂ + 0.0844 𝐞₁₃ + -0.0171 𝐞₂₃

In [90]:
#6.38
r=2
f_(Ar)(x), (x-n).sw(Ar(x))/normsq(x-n)/(-l(x))**r

(-0.685 𝐞₁₂ + -0.487 𝐞₁₃ + 0.0986 𝐞₂₃, -0.685 𝐞₁₂ + -0.487 𝐞₁₃ + 0.0986 𝐞₂₃)

In [91]:
#6.39
s = 1
PB = lambda x: P(B(x), Ix(x))
Bs = lambda x: PB(x).grade(s)
f_(Ar)(x) * f_(Bs)(x), (x-n).sw(Ar(x)*Bs(x))/normsq(x-n)/(-l(x))**(r+s)

(0.0827 𝐞₁ + -1.98 𝐞₂ + -1.39 𝐞₃ + -1.11e-16 𝐞₁₂₃,
 0.0827 𝐞₁ + -1.98 𝐞₂ + -1.39 𝐞₃)

In [92]:
#6.41
A1r = f_(Ar)
fa(A1r)(x), (x-n).sw(A1r(x))/normsq(x-n)/(-l(x))**r, f_(A1r)(x),

(0.394 𝐞₁₂ + -1.52 𝐞₁₃ + 0.191 𝐞₂₃,
 0.394 𝐞₁₂ + -1.52 𝐞₁₃ + 0.191 𝐞₂₃,
 -0.466 𝐞₁₂ + -0.331 𝐞₁₃ + 0.067 𝐞₂₃)